In [4]:
import numpy as npi
import seaborn as sns
import cv2, os, glob, datetime
import matplotlib.pyplot as plt
from keras import layers, Input
from tensorflow.keras.optimizers import SGD
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications import InceptionV3
from keras.models import Sequential, Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation, Dropout, GlobalAveragePooling2D

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50  # You can switch to other models like VGG16, Xception, etc.

inp = 299  
model = InceptionV3(weights="imagenet", include_top=False, input_shape=(inp, inp, 3))

# Folder paths
folders = glob.glob(r"D:\adnan blood detection2\*")

# Initialize lists for storing images and labels
img_list = []
label_list = []

# Loop through the folders and read images
for folder in folders:
    print(f"Processing folder: {folder}")
    
    # Read both .png and .jpg images
    image_paths = glob.glob(folder + r"/*.jpg") + glob.glob(folder + r"/*.png")
    
    if not image_paths:
        print(f"No images found in {folder}")
    
    for img in image_paths:
        n = cv2.imread(img)
        if n is None:
            print(f"Failed to read image: {img}")
            continue
        
        class_num = folders.index(folder)  # Assign label based on folder index
        label_list.append(class_num)
        
        # Resize image to the required size for ResNet50 (224x224)
        resized = cv2.resize(n, (inp, inp), interpolation=cv2.INTER_AREA)
        img_list.append(resized)

# Check if img_list and label_list are populated
if not img_list or not label_list:
    raise ValueError("No images found. Check the paths and image files.")

# Split data into training (70%) and testing (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(img_list, label_list, test_size=0.3, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print the shapes of the datasets
print("training_set", X_train.shape)
print("training_set", y_train.shape)
print("validation_set", X_valid.shape)
print("validation_set", y_valid.shape)
print("test_set", X_test.shape)
print("test_set", y_test.shape)

# Count the number of images for each class in the entire dataset
unique_classes, class_counts = np.unique(label_list, return_counts=True)

# Create a bar chart to visualize the number of images for each class
plt.figure(figsize=(10, 6))
plt.bar(unique_classes, class_counts, color='skyblue')

# Labeling the chart
plt.xlabel('Class Index')
plt.ylabel('Number of Images')
plt.title('Number of Images in Each Class')

# Add labels on top of each bar
for i, count in enumerate(class_counts):
    plt.text(i, count + 10, str(count), ha='center', fontsize=12)

# Show the plot
plt.show()


In [ ]:
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import ResNet50  # You can switch to other models like VGG16, Xception, etc.

# Initialize the model
inp = 224  # ResNet50 requires input shape (224, 224, 3)
model = ResNet50(weights="imagenet", include_top=False, input_shape=(inp, inp, 3))

# Folder paths
folders = glob.glob(r"D:\adnan blood detection2\*")

# Initialize lists for storing images and labels
img_list = []
label_list = []

# Loop through the folders and read images
for folder in folders:
    print(f"Processing folder: {folder}")
    
    # Read both .png and .jpg images
    image_paths = glob.glob(folder + r"/*.jpg") + glob.glob(folder + r"/*.png")
    
    if not image_paths:
        print(f"No images found in {folder}")
    
    for img in image_paths:
        n = cv2.imread(img)
        if n is None:
            print(f"Failed to read image: {img}")
            continue
        
        class_num = folders.index(folder)  # Assign label based on folder index
        label_list.append(class_num)
        
        # Resize image to the required size for ResNet50 (224x224)
        resized = cv2.resize(n, (inp, inp), interpolation=cv2.INTER_AREA)
        img_list.append(resized)

# Check if img_list and label_list are populated
if not img_list or not label_list:
    raise ValueError("No images found. Check the paths and image files.")

# Split data into training (70%) and testing (30%) sets
X_train, X_temp, y_train, y_temp = train_test_split(img_list, label_list, test_size=0.3, random_state=1)
X_valid, X_test, y_valid, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=1)

# Convert lists to numpy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_test = np.array(X_test)
y_test = np.array(y_test)

# Print the shapes of the datasets
print("training_set", X_train.shape)
print("training_set", y_train.shape)
print("validation_set", X_valid.shape)
print("validation_set", y_valid.shape)
print("test_set", X_test.shape)
print("test_set", y_test.shape)
print("Train_Folder", len(X_train))
print("validation_Folder", len(X_valid))
print("Test_Folder", len(X_test))


In [ ]:
inputs =Input((inp, inp, 3))
X = model(inputs)
flat1 = GlobalAveragePooling2D()(X)
x3=layers.GlobalMaxPooling2D()(X)
x1=layers.Dense(100, activation='relu')(flat1)
x1=layers.Dense(50, activation='relu')(x1)
x1=layers.BatchNormalization()(x1)

x2=layers.Conv2D(filters = 64,kernel_size = (1,1), activation='relu', padding='same')(X)
x2=layers.Conv2D(filters = 64,kernel_size = (7,7), activation='relu', padding='same')(x2)
x2=layers.Conv2D(filters = 64,kernel_size = (1,1), activation='relu', padding='same')(x2)

x2=layers.GlobalAveragePooling2D()(x2)
x2=layers.BatchNormalization()(x2)

BAM=layers.concatenate([x1, x2])
BAM=layers.BatchNormalization()(BAM)
BAM=layers.concatenate([x3, BAM])

F=layers.Dense(150, activation='relu')(BAM)
F=layers.BatchNormalization()(F)
output = Dense(3,activation='softmax')(F)
model = Model(inputs=inputs, outputs=output)

model.summary()

In [ ]:
batch_size = 16
epochs = 30
opt = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(X_valid,y_valid))
model.save("adxnan1.h5")


In [ ]:
batch_size = 16
epochs = 30
opt = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history=model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,verbose=1,validation_data=(X_valid,y_valid))
model.save("adxnan1.h5")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import  confusion_matrix
import matplotlib.pyplot as plot
import seaborn as sn
import pandas as pd
import seaborn as sns
from sklearn.metrics import  confusion_matrix
from sklearn.metrics import classification_report 
import numpy as np
plt.figure(figsize=(30,30))
sns.set(font_scale=1.0)

f, ax = plt.subplots()
ax.plot([None] + history.history['accuracy'])
ax.plot([None] + history.history['val_accuracy'])
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train acc', 'Val acc'], loc = 0)
ax.set_title('Training/Validation acc per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Accuracy')
f, ax = plt.subplots()
ax.plot([None] + history.history['loss'])
ax.plot([None] + history.history['val_loss'])
# Plot legend and use the best location automatically: loc = 0.
ax.legend(['Train loss', "Val loss"], loc = 1)
ax.set_title('Training/Validation Loss per Epoch')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
plt.show()


y_pred=model.predict(X_test)
y_pred=np.argmax(y_pred, axis=1)

target_names = ["Scene 1","Scene 2","Scene 3"] 
cm = confusion_matrix(y_test, y_pred)
print("***** Confusion Matrix *****")
print(cm)
print("***** Classification Report *****")
print(classification_report(y_test, y_pred, target_names=target_names))
classes=3
con = np.zeros((classes,classes))
for x in range(classes):
    for y in range(classes):
        con[x, y] = cm[x, y] / np.sum(cm[x, :]) * 100  # Calculate percentage

plt.figure(figsize=(15, 12))
df = sns.heatmap(con, annot=True, fmt='.2f', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
df.figure.savefig("Model.png")
plt.show()
print('\nTesting loss: {:.4f}\nTesting accuracy: {:.4f}'.format(*model.evaluate(X_test, y_test)))